In [6]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
text = (open(r"C:\Users\Nathan Lee\Workspace\EssayWriter\training_data.txt", encoding="utf-8").read())
text = text.lower()
print(text)

how to cope with the fact that you are a bag of meat sitting on a rock in outer space and that someday you will die

we can not prove the past really happened.  everything can be proved with “last thursdayism”, the philosophical thought that the universe was created last thursday, where everything that is here and that you remember was created by an omnipotent being, putting on the grand illusion of reality.  we ,of course, can not think this way.  nothing would have a point, and everything would feel shallow.  our relationships with others have to mean something, or our lives will not mean anything.  relationships form a large part of our identities and without them, we would be lost just like if we were not sure if anything is real, because we would not understand the point of doing anything.  cormac mccarthy discusses similar topics in his novel all the pretty horses.  the novel discusses ideas about loss and its connection with relationships and identity.  the novel is set after ww